# `pandas_bridge`

The `evo.tools.pandas_bridge` module converts:
* `evo.core.trajectory.PosePath3D`
* `evo.core.trajectory.PoseTrajectory3D`
* `evo.core.result.Result`

to Pandas dataframes.

Load some required modules for this demo:

In [ ]:
# magic plot configuration
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%matplotlib notebook

from IPython.display import display

Load Pandas and `pandas_bridge`:

In [ ]:
import pandas as pd
from evo.tools import pandas_bridge
from evo.tools import file_interface

## Trajectories

In [ ]:
# load a trajectory from a ROS bag...
from rosbags.rosbag1 import Reader as Rosbag1Reader
with Rosbag1Reader("../test/data/ROS_example.bag") as reader:
    traj = file_interface.read_bag_trajectory(reader, "S-PTAM")

In [ ]:
# ...or from a KITTI file...
traj = file_interface.read_kitti_poses_file("../test/data/KITTI_00_gt.txt")

In [ ]:
# ...or from a TUM file...
traj = file_interface.read_tum_trajectory_file("../test/data/fr2_desk_ORB.txt")

**Convert a trajectory to a dataframe:**

In [ ]:
traj_df = pandas_bridge.trajectory_to_df(traj)

**Some examples for what you can do with it:**

In [ ]:
print("First entries of the dataframe:")
display(traj_df.head())

print("Some statistics of the dataframe:")
display(traj_df.describe())

print("A plot:")
traj_df[["x", "y", "z"]].plot(kind="line", subplots=True)

# Results

In [ ]:
# generate some result files
import subprocess as sp
cmd_1 = "evo_ape kitti ../test/data/KITTI_00_gt.txt ../test/data/KITTI_00_ORB.txt --save_results ../test/data/res1.zip --no_warnings"
cmd_2 = "evo_ape kitti ../test/data/KITTI_00_gt.txt ../test/data/KITTI_00_SPTAM.txt --save_results ../test/data/res2.zip --no_warnings"
sp.call(cmd_1.split(" "))
sp.call(cmd_2.split(" "))

Load results:

In [ ]:
result_1 = file_interface.load_res_file("../test/data/res1.zip")
result_2 = file_interface.load_res_file("../test/data/res2.zip")

Convert results into individual dataframes and concatenate them:

In [ ]:
result_df_1 = pandas_bridge.result_to_df(result_1)
result_df_2 = pandas_bridge.result_to_df(result_2)

result_df = pd.concat([result_df_1, result_df_2], axis="columns")
display(result_df)

**Some examples for what you can do with it:**

In [ ]:
display(result_df.loc["stats"])

exclude = result_df.loc["stats"].index.isin(["sse"])  # don't plot sse
result_df.loc["stats"][~exclude].plot(kind="bar")